# Laplacian variance

* A sharp image has lots of fast intensity changes (edges).
* The Laplacian operator highlights those rapid changes.
* If you take the **variance** of the Laplacian response, you get a single number: high for sharp images (many strong edges), low for blurry ones (edges smeared out).

# What you compute

1. Convert to grayscale.
2. Apply Laplacian (2nd derivative) → an image of edge “strength”.
3. Compute the **variance** (σ²) of that Laplacian image.
4. Compare σ² to a threshold T:

   * σ² < T → likely blurry
   * σ² ≥ T → likely sharp

There’s no universal T; you pick it from your data (see tips at the end).

---

# Python (OpenCV)

```python
import cv2 as cv
import numpy as np

def variance_of_laplacian(img_bgr) -> float:
    gray = cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)
    # Use ksize=3; CV_64F to avoid clipping negatives
    lap = cv.Laplacian(gray, ddepth=cv.CV_64F, ksize=3, scale=1, delta=0)
    return lap.var()

img = cv.imread("frame.png")
score = variance_of_laplacian(img)
print("Laplacian variance:", score)

T = 150.0  # example; tune for your dataset
print("Blurry" if score < T else "Sharp")
```

If your images are already single-channel 16-bit (e.g., `CV_16UC1`), **don’t** downscale to 8-bit before the Laplacian; convert to float and keep dynamic range:

```python
gray16 = cv.imread("depth.png", cv.IMREAD_UNCHANGED)  # CV_16UC1
grayf  = gray16.astype(np.float32)
lap = cv.Laplacian(grayf, cv.CV_32F, ksize=3)
score = lap.var()
```

---

# C++ (OpenCV)

```cpp
#include <opencv2/opencv.hpp>
#include <iostream>

double varianceOfLaplacian(const cv::Mat& bgr)
{
    cv::Mat gray, lap, lap64;
    if (bgr.channels() == 3) {
        cv::cvtColor(bgr, gray, cv::COLOR_BGR2GRAY);
    } else {
        gray = bgr;
    }
    cv::Laplacian(gray, lap, CV_64F, 3, 1, 0, cv::BORDER_DEFAULT);

    cv::Scalar mean, stddev;
    cv::meanStdDev(lap, mean, stddev);
    return stddev[0] * stddev[0]; // variance
}

int main()
{
    cv::Mat img = cv::imread("frame.png");
    double score = varianceOfLaplacian(img);
    std::cout << "Laplacian variance: " << score << "\n";
    double T = 150.0; // tune this
    std::cout << ((score < T) ? "Blurry\n" : "Sharp\n");
    return 0;
}
```

For `CV_16UC1`:

```cpp
cv::Mat depth16 = cv::imread("depth.png", cv::IMREAD_UNCHANGED);
cv::Mat depth32f;
depth16.convertTo(depth32f, CV_32F);       // preserve scale
cv::Mat lap;
cv::Laplacian(depth32f, lap, CV_32F, 3);
cv::Scalar m, s; cv::meanStdDev(lap, m, s);
double var = s[0]*s[0];
```

---

# Practical tips & pitfalls

* **Threshold selection:** gather a few dozen “sharp” and “blurry” samples, compute scores, plot their distributions, and choose T (e.g., a value between the clusters or using the 5–10th percentile of the sharp class as a cutoff). Resolution, noise, and content affect absolute values.
* **Noise & compression:** strong noise or JPEG block artifacts can inflate the score. A light pre-blur (e.g., bilateral or Gaussian σ=0.5–1.0) can stabilize results.
* **Scale/bit depth:** scores grow with intensity scale and scene texture. Keep preprocessing consistent (bit depth, contrast). If you must compare across different exposures/bit-depths, normalize:

  * Divide the Laplacian by 255 (8-bit) or 65535 (16-bit) before variance, **or**
  * Use a **normalized focus measure**, e.g. `var(Lap) / mean(gray)^2`.
* **Avoid borders:** derivatives are sensitive at borders. You can crop a small border (e.g., 8–16 px) before computing.
* **ROI-based:** if only a region matters (e.g., subject center), compute on that ROI to avoid background bias.
* **Multi-scale option:** take max/mean of variance computed with ksize 1,3,5 (helps when blur varies).
* **Alternatives:** Tenengrad (variance/energy of Sobel magnitude), Sum Modified Laplacian (SML), or frequency-domain measures (high-frequency energy).

---

# A tiny helper to auto-pick T (Python)

```python
def pick_threshold(sharp_scores, blur_scores):
    import numpy as np
    # Simple heuristic: midpoint of medians
    return 0.5 * (np.median(sharp_scores) + np.median(blur_scores))
```

That’s it: compute Laplacian → variance → compare to a tuned threshold. It’s simple, fast, and works well as a first-pass blur detector.

---

## 1. Do we need a Gaussian first?

* **Not required.**
  The basic Laplacian variance method just applies the Laplacian to the grayscale image.
* **But often useful.**
  Real images have noise. Noise produces sharp pixel-to-pixel jumps → Laplacian will treat them as edges → variance gets inflated.
  A small Gaussian blur (`σ ≈ 1.0`, `ksize=3`) smooths noise while keeping meaningful edges.
  This is why “Laplacian of Gaussian (LoG)” is also common in vision.

So: Gaussian = optional pre-step for robustness.

---

## 2. How do we calculate the Laplacian?

Yes, with **convolution kernels** (second derivatives).

The continuous Laplacian of an image $I(x,y)$ is:

$$
\nabla^2 I = \frac{\partial^2 I}{\partial x^2} + \frac{\partial^2 I}{\partial y^2}
$$

In discrete form (on a pixel grid), OpenCV approximates this by convolving with kernels.
A common 3×3 kernel is:

$$
\begin{bmatrix}
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0
\end{bmatrix}
$$

or the 8-neighbor version:

$$
\begin{bmatrix}
1 & 1 & 1 \\
1 & -8 & 1 \\
1 & 1 & 1
\end{bmatrix}
$$

* Both approximate the Laplacian operator (just scaled differently).
* OpenCV’s `cv::Laplacian` internally uses `cv::Sobel` to compute 2nd derivatives and adds them.

So mathematically, the Laplacian image is:

$$
L(i,j) = \sum_{u,v} K(u,v) \cdot I(i+u, j+v)
$$

where $K$ is the Laplacian kernel.

---

## 3. How do we calculate the variance of the Laplacian?

Right: the Laplacian image $L$ is just a matrix of values (positive and negative).
Variance is a **statistical measure of spread**:

$$
\mathrm{Var}(L) = \frac{1}{N} \sum_{i=1}^N \left(L_i - \mu \right)^2
$$

where

* $N$ = number of pixels
* $L_i$ = Laplacian value at pixel $i$
* $\mu = \frac{1}{N} \sum_i L_i$ is the mean Laplacian value

This is exactly what `numpy.var` or OpenCV’s `meanStdDev` does.

So in practice:

```python
lap = cv2.Laplacian(gray, cv2.CV_64F)
score = lap.var()   # variance of all pixels
```

---

✅ To summarize:

1. (Optional) Gaussian blur to suppress noise.
2. Apply Laplacian → convolution with kernels (OpenCV does it internally).
3. Compute variance of all pixel values in Laplacian image → a single scalar score.

* **Sharp image** → edges strong → Laplacian values large and diverse → **high variance**
* **Blurred image** → edges smoothed → Laplacian values close to zero → **low variance**

---